In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import re
from io import StringIO
import PyPDF2
import pandas as pd
from datetime import datetime 
import datetime as dt
from timeit import default_timer as timer
pd.set_option('display.max_rows', 500)

In [2]:
folder = "../Dec2020"
resultFolder = "Dec2020_Result"
file = "12.01 3207971832_2020-December_eStatement.pdf"
path = folder + '/' + file
excelFile = '../' + resultFolder + '/' + re.sub('\.', '_', '01_12 ' + file.split('-')[0].split(' ')[1]) + '.xlsx'
excelFile

'../Dec2020_Result/01_12 3207971832_2020.xlsx'

In [3]:
date = re.sub('\.', '/', file.split(' ')[0])
previousDate = (datetime.strptime(date, "%m/%d") - dt.timedelta(days=1)).strftime('%d/%m')
#for nov
date = datetime.strptime(date, "%m/%d").strftime('%d/%m')
date, previousDate

('01/12', '30/11')

In [4]:
def convertPdfToTxt(path):
    startTime = timer()
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    #codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages = maxpages,caching = caching, check_extractable = True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    endTime = timer()
    print("Total time: %0.4fs" % (endTime - startTime))
    return text

text = convertPdfToTxt(path)

#extract no pages
noPage = len(re.findall("NIAGADEBITKREDITBAKIDATETRANSACTIONDEBITCREDITBALANCE", text))

noPage

Total time: 105.2384s


1142

In [5]:
accountInfoDict = {}
accountCol = ["Nombor Akaun / Account Number", 'Jenis Akaun / Account Type',
             'Tarikh Penyata / Statement Date', 'RINGKASAN / SUMMARY']
for no, desc in enumerate(accountCol):
    if no != len(accountCol)-1:
        descIndex = text.index(desc)
        descLen = len(desc)
        nextDescIndex = text.index(accountCol[no + 1])
        result = text[descIndex + descLen: nextDescIndex]
        accountInfoDict[desc] = result
accountInfoDict

{'Nombor Akaun / Account Number': '3207971832',
 'Jenis Akaun / Account Type': 'RM Plus Current Account',
 'Tarikh Penyata / Statement Date': '01 Dec 2020'}

In [6]:
def extractText(desc, startStr, endStr):
    try:
        startStrIndex = desc.index(startStr)
        startStrLen = len(startStr)
        endStrIndex = desc.index(endStr)         
        if startStr == '':
            result = desc[:endStrIndex]
            return result
        elif endStr!='':
            result = desc[startStrIndex + startStrLen:endStrIndex]
            remainDesc = desc[endStrIndex:]
            return result, remainDesc
        else:
            result = desc[startStrIndex + startStrLen:]
            return result
        
    except Exception as e:
#         print(e)
        print("Failed to extract text")

def extractAmount(desc, greedy = True):
#     greedyPattern = '([\d\.\,]+)'
    greedyPattern = '([\d\.\,]+\.\d{2})'
    notGreedyPattern = '([\d\.\,]+?\.\d{2}?)'
    try:
        if greedy:
            result = re.search(greedyPattern, desc).group(1)
            return result
        else:
            result = re.search(notGreedyPattern, desc).group(1)
            return result
    except Exception as e:
#         print(e)
        print("Wrong number pattern")

In [7]:
summaryDict = {}
summaryCol = ['Baki Penutup / Closing Balance',
              'Jumlah Debit / Total Debits', 'Bil. Debit / No. of Debits',
              'Jumlah Kredit / Total Credits', 'Bil. Kredit / No. of Credits']
#extract series of number 
numberSeries = extractText(text, summaryCol[4], 'Muka Surat 1')[0]

#extract closing balance amount
closingBalanceAmount = extractAmount(numberSeries, greedy = False)
summaryDict[summaryCol[0]] = closingBalanceAmount
numberSeries = re.sub(closingBalanceAmount, '', numberSeries)

#extract debit amount
debitAmount = extractAmount(numberSeries, greedy = False)
summaryDict[summaryCol[1]] = debitAmount
numberSeries = re.sub(debitAmount, '', numberSeries)

#credit no credit
noCredit = re.sub(extractAmount(numberSeries, greedy = False), '', numberSeries)
summaryDict[summaryCol[4]] = noCredit
numberSeries = re.sub(noCredit + '$', '', numberSeries)

tempNumberSeries = numberSeries
creditAmount1 = re.search('(\d{2}\,(\d+\,)*\d+\.\d{2})', numberSeries).group(1)
noDebit1 = re.sub(creditAmount1, '', tempNumberSeries)
creditAmount2 = re.search('(\d{1}\,(\d+\,)*\d+\.\d{2})', numberSeries).group(1)
noDebit2 = re.sub(creditAmount2, '', tempNumberSeries)

noDebitList = [noDebit1, noDebit2]
creditAmountList = [creditAmount1, creditAmount2]
noDebitList, creditAmountList

(['12816', '128161'], ['12,298,806.51', '2,298,806.51'])

In [8]:
def returnTableList(no):
    tempList = []
    textCopy = text
    for i in range(1, no + 1):
#     for i in range(1, 3):
        table, remain = extractText(textCopy, 'Page %s of %s'%(i, no), 'Penyata ini dicetak melalui komputer.')
        index = remain.index('Penyata ini dicetak melalui komputer.')
        termLen = len('Penyata ini dicetak melalui komputer.')
        textCopy = remain[index + termLen:]
        table = re.sub("TARIKHURUS NIAGADEBITKREDITBAKIDATETRANSACTIONDEBITCREDITBALANCE", '', table)
        tempList.append(table)
    return tempList

tableList = returnTableList(noPage)

len(tableList)

1142

In [49]:
#extract first line of transaction

#trx date, cheque no., ref1, ref2, ref3, ref4, credit amount, debit amount, balance
resultList = []

#extract content of first table from text
table = tableList[0]

#extract balance amount
balanceAmount, tempText = extractText(table, "Balance From Last Statement", date)

#store record of balance from last statement
# resultList.append([previousDate, '','', '','', "Balance From Last Statement", '', '', balanceAmount])
#trx date, ref1, credit amount, debit amount, balance
resultList.append([previousDate, "Balance From Last Statement", '', '', balanceAmount])

In [31]:
def extractText(desc, startStr, endStr):
    try:
        startStrIndex = desc.index(startStr)
        startStrLen = len(startStr)
        endStrIndex = desc.index(endStr)         
        if startStr == '':
            result = desc[:endStrIndex]
            return result
        elif endStr!='':
            result = desc[startStrIndex + startStrLen:endStrIndex]
            remainDesc = desc[endStrIndex:]
            return result, remainDesc
        else:
            result = desc[startStrIndex + startStrLen:]
            return result
        
    except Exception as e:
#         print(e)
        print("Failed to extract text")

In [50]:
#extract amountList and descList
#steps: 
#1. extract first and second number, store first number in descList
#2. extract desc between first and second number
#3. store desc in descList
#repeat 1 till 3 until closing balance's line

amountList = []
descList = []
closingLine = 0

# for no, table in enumerate([tableList[0]]):
for no, table in enumerate(tableList):
    if no == 0:
        tempText = tempText
    else:
        tempText = table
    # for i in range(3):
    while closingLine == 0:
        #remove date first
        if re.search(date, tempText):
            tempText = re.sub(date, '', tempText)
        #remove RMB @1.55
        tempText = re.sub("RMB.*@ \d\.\d+", '', tempText)
        if no in [539, 1010]:
        #remove 9.10726135944E11: table 976
            tempText = re.sub("9.10726135944E11", '', tempText)
        #extract amount first
        tempAmount1 = extractAmount(tempText)
        #store amount
        amountList.append(tempAmount1)
        #temporary subbed text to get second amount
        tempSubText = re.sub(tempAmount1, '', tempText)
        tempAmount2 = extractAmount(tempSubText)
        tempDesc = extractText(tempSubText, '', tempAmount2)
        
        #if Balance C/F, last line of transaction for that page
        if re.search(r"Balance C/F", str(tempDesc)):
            tempDesc = re.sub(r"Balance C/F", "", str(tempDesc))
            amountList.append(tempAmount2)
            descList.append(tempDesc)
            descList.append("Balance C/F")
            break
            
        #if Balance B/F, first line of transaction for that page
        elif re.search(r"Balance B/F", str(tempDesc)):
            tempDesc = re.sub(r"Balance B/F", "", str(tempDesc))
            descList.append("Balance B/F")
            tempAmount1 = extractAmount(re.sub(date, '', tempText), greedy = False)
            amountList[-1] = tempAmount1
            tempAmount1Index = tempText.index(tempAmount1)
            tempAmount1Len = len(tempAmount1)
            tempText = tempText[tempAmount1Index+tempAmount1Len:]
            
        #if last line for whole file
        elif re.search("Closing Balance In This Statement", str(tempDesc)):
            closingAmount = tempAmount2
            closingAmountIndex = tempSubText.index(closingAmount)
            lastDesc = tempSubText[:closingAmountIndex]
            #sub closing amount with '' in lastDesc
            lastDesc = re.sub("Closing Balance In This Statement", '', lastDesc)
            descList.append(lastDesc)

            #store last line - amount and desc
            amountList.append(closingAmount)
            descList.append("Closing Balance In This Statement")
#             closingLine = 1
            break
        #extracted all ady
        elif not tempAmount2:
#             closingLine = 1
            break
        else:
            tempText = re.sub('^' + re.escape(tempDesc), '', tempSubText)
            descList.append(tempDesc)

In [48]:
tableList[539]

'01/12Balance B/F29,974,969.61EU REFUND 24112020 GOH BEE YIN650.0029,974,319.61DR-ECP 0088973207971832PM01122099 EU REFUNDEU REFUND 24112020 LEE GAIK PENG2,391.0029,971,928.61DR-ECP 0088993207971832PM01122099 EU REFUNDEU REFUND 24112020 ONG BEE CHOO70.0029,971,858.61DR-ECP 0089013207971832PM01122099 EU REFUNDEU REFUND 24112020 LIM JIA HOU599.0029,971,259.61DR-ECP 0089033207971832PM01122099 EU REFUNDEU REFUND 24112020 LEONG HENG WAN558.0029,970,701.61DR-ECP 0089053207971832PM01122099 EU REFUNDEU REFUND 24112020 CHAN MAU CHEN4,077.0029,966,624.61DR-ECP 0089073207971832PM01122099 EU REFUNDEU REFUND 24112020 CHAN CHENG MUN3,539.0029,963,085.61DR-ECP 0089093207971832PM01122099 EU REFUNDEU REFUND 24112020 9.10726135944E11953.0029,962,132.61DR-ECP 0089113207971832PM01122099 EU REFUNDEU REFUND 24112020 TINA HUONG1,700.0029,960,432.61DR-ECP 0089133207971832PM01122099 EU REFUNDEU REFUND 24112020 GOH YONG SHENG1,119.0029,959,313.61DR-ECP 0089153207971832PM01122099 EU REFUNDEU REFUND 24112020 CHIE

In [47]:
tableList[1010]

'01/12Balance B/F28,716,603.11FEE 3207971832PM01122099EU REFUND EU REFUND 24112020CHUNG SEN SHEN0.1028,716,603.01HANDLING CHRG 014680FEE 3207971832PM01122099EU REFUND EU REFUND 24112020CHEONG MUN HOU0.1028,716,602.91HANDLING CHRG 014681FEE 3207971832PM01122099EU REFUND EU REFUND 241120209.10726135944E110.1028,716,602.81HANDLING CHRG 014682FEE 3207971832PM01122099EU REFUND EU REFUND 24112020TINA HUONG0.1028,716,602.71HANDLING CHRG 014683FEE 3207971832PM01122099EU REFUND EU REFUND 24112020GOH YONG SHENG0.1028,716,602.61HANDLING CHRG 014684FEE 3207971832PM01122099EU REFUND EU REFUND 24112020CHIENG CHING YIN0.1028,716,602.51HANDLING CHRG 014685FEE 3207971832PM01122099EU REFUND EU REFUND 24112020LAW BENG AI0.1028,716,602.41HANDLING CHRG 014686FEE 3207971832PM01122099EU REFUND EU REFUND 24112020SOH CHIN ZI0.1028,716,602.31HANDLING CHRG 014687FEE 3207971832PM01122099EU REFUND EU REFUND 24112020KHOO KING SOON0.1028,716,602.21HANDLING CHRG 014688FEE 3207971832PM01122099EU REFUND EU REFUND 24112

In [51]:
def splitAmount(desc):
    try:
        #remove first . before num if exists
        desc = re.sub('^\.', '', desc)
        #remove date pattern
        desc = re.sub('^\d{1,2}\.\d{1,2}\.\d{4}', '', desc)
        amountPattern = '(.*?\.\d{2}?)'
        result = re.search(amountPattern, desc).group(1)
        desc2 = re.sub('^' + result, '', desc)
        return result, desc2
    except Exception as e:
        print(e)
        print("Wrong amount pattern")

splitAmountList = [splitAmount(i) for i in amountList]

In [52]:
def checkCreditDebit(amountTuple, previousBalance):
    #exception:
    amount1 = float(re.sub(',', '', amountTuple[0]))
#     currentBalance = re.sub('^\.', '', amountTuple[1])
    currentBalance = amountTuple[1]
#     print(currentBalance)
    if currentBalance !='':
        currentBalance = float(re.sub(',', '', currentBalance))
        previousBalance = float(re.sub(',', '', previousBalance))
        if currentBalance - previousBalance == amount1:
            amountTuple = ('', round(amount1, 2), currentBalance)
        elif previousBalance - currentBalance == amount1:
            amountTuple = (round(amount1, 2), '', currentBalance)
        #if amount 1 is wrong
        elif currentBalance > previousBalance:
            credit  = currentBalance - previousBalance
            amountTuple = ('', round(credit,2), currentBalance)
        elif previousBalance > currentBalance:
            debit  = previousBalance - currentBalance
            amountTuple = (round(debit,2), '', currentBalance)            
    else:
        amountTuple = ('', '', amount1)
    return amountTuple, str(amountTuple[2])

#regex pattern based on transDesc
transTypePatternDict = {'INSTANT TRSF CR':'(.*CR)\s*(\d{6})(.+)',
                       'TSFR FUND DR-ATM/EFT':'(TSFR FUND DR-ATM/EFT)\s*(\d{6})(.+)'}

def splitStr(desc):
    foundPattern = 0
    for transType in transTypePatternDict:
        if re.search(transTypePatternDict[transType], desc):
            pattern = transTypePatternDict[transType]
            transType = re.search(pattern, desc).group(1)
            chequeNo = re.search(pattern, desc).group(2)
            ref1  = re.search(pattern, desc).group(3)
            foundPattern = 1
            finalResult = [chequeNo, transType, ref1] + ['']*2
            break
    if foundPattern == 0:
        finalResult = ['']*2 + [desc] + ['']*2
    #[chequeNo, transactionDescType, ref1, ref2, ref3]
    return finalResult

In [28]:
tableList[539]

'01/12Balance B/F29,974,969.61EU REFUND 24112020 GOH BEE YIN650.0029,974,319.61DR-ECP 0088973207971832PM01122099 EU REFUNDEU REFUND 24112020 LEE GAIK PENG2,391.0029,971,928.61DR-ECP 0088993207971832PM01122099 EU REFUNDEU REFUND 24112020 ONG BEE CHOO70.0029,971,858.61DR-ECP 0089013207971832PM01122099 EU REFUNDEU REFUND 24112020 LIM JIA HOU599.0029,971,259.61DR-ECP 0089033207971832PM01122099 EU REFUNDEU REFUND 24112020 LEONG HENG WAN558.0029,970,701.61DR-ECP 0089053207971832PM01122099 EU REFUNDEU REFUND 24112020 CHAN MAU CHEN4,077.0029,966,624.61DR-ECP 0089073207971832PM01122099 EU REFUNDEU REFUND 24112020 CHAN CHENG MUN3,539.0029,963,085.61DR-ECP 0089093207971832PM01122099 EU REFUNDEU REFUND 24112020 9.10726135944E11953.0029,962,132.61DR-ECP 0089113207971832PM01122099 EU REFUNDEU REFUND 24112020 TINA HUONG1,700.0029,960,432.61DR-ECP 0089133207971832PM01122099 EU REFUNDEU REFUND 24112020 GOH YONG SHENG1,119.0029,959,313.61DR-ECP 0089153207971832PM01122099 EU REFUNDEU REFUND 24112020 CHIE

In [27]:
amountList[540]

'264.8937,021,875.35'

In [24]:
amountList[7680]

'11953.0029,962,132.61'

In [46]:
descList[13546]

'HANDLING CHRG 014682FEE 3207971832PM01122099EU REFUND EU REFUND 24112020TINA HUONG'

In [23]:
splitAmountList2[7680]

('', 29962123.51, 29962132.61)

In [53]:
#categorize one amount as credit/debit
splitAmountList2 = []
for no, i in enumerate(splitAmountList):
    if no == 0:
        resultTuple, previousBalanceAmount = checkCreditDebit(i, balanceAmount)
        splitAmountList2.append(resultTuple)
    else:
        try:
            resultTuple, previousBalanceAmount = checkCreditDebit(i, previousBalanceAmount)
            splitAmountList2.append(resultTuple)
        except:
            print(no)
            print(i)
# splitStrList = [splitStr(i) for i in descList]

# #column names
# colList = ['Trn. Date', 'Cheque No/Ref No', 'Transaction Description',
#            'Debit Amount', 'Credit Amount', 
#            'Reference 1', 'Reference 2', 'Reference 3', 'Balance']

# for no, finalStrList in enumerate(splitStrList):
#     tempList =  [date] + [i for i in finalStrList[:2]]
#     tempList += [i for i in splitAmountList2[no][:2]] + [i for i in finalStrList[2:]] + [splitAmountList2[no][-1]]
#     resultList.append(tempList)
    
colList = ['Trn. Date','Reference 1',
           'Debit Amount', 'Credit Amount', 'Balance']
for no, finalStrList in enumerate(descList):
    tempList =  [date] + [finalStrList]
    tempList +=  [i for i in splitAmountList2[no]]
    resultList.append(tempList)

In [54]:
df = pd.DataFrame(resultList, columns = colList)
df

,Trn. Date,Reference 1,Debit Amount,Credit Amount,Balance
0,30/11,Balance From Last Statement,,,"25,487,020.34"
1,01/12,SI CR FROM 3999XXXXXX,,1.19489e+07,3.7436e+07
2,01/12,DEP-CASH AT LHA,,18160,3.74541e+07
3,01/12,RTN ITEM 092166IBG RTN ITEM 20201201 LTCS29201...,,19.6,3.74541e+07
4,01/12,RTN ITEM 092167IBG RTN ITEM 20201201 LTCS29201...,,473.92,3.74546e+07
...,...,...,...,...,...
15185,01/12,INSTANT TRSF DR 573029 TAN BEE LAYMERCHANT RED...,31,,2.84416e+07
15186,01/12,TSFR FUND DR-ATM/EFT 5730544417XXXXXX TAN LIAN...,5302,,2.84363e+07
15187,01/12,INSTANT TRSF DR 573055MAX IVER SAMUEL COP REFUND,900,,2.84354e+07
15188,01/12,INSTANT TRSF DR 573087 PDA FACTORMERCHANT REDE...,1831.71,,2.84336e+07


In [42]:
max([i for i in df['Debit Amount'] if type(i)!=str])

212403606.29

In [45]:
df.query("`Debit Amount`==212403606.29")

,Trn. Date,Reference 1,Debit Amount,Credit Amount,Balance
13547,01/12,HANDLING CHRG 014682FEE 3207971832PM01122099EU...,2.12404e+08,,2.87166e+07


In [22]:
df.query("`Credit Amount`==29962123.51")['Reference 1'].values

array(['DR-ECP 0089113207971832PM01122099 EU REFUNDEU REFUND 24112020 TINA HUONG'],
      dtype=object)

In [55]:
df.loc[df.shape[0]-1]['Balance']

28433583.71

In [56]:
sum([i for i in df['Debit Amount'] if i!=''])

9352243.139998108

In [57]:
trueNoDebit = df.query("`Debit Amount`!=''").shape[0]
trueNoDebit

12816

In [58]:
sum([i for i in df['Credit Amount'] if i!=''])

12298806.509999998

In [59]:
#confirm no debit
noDebitIndex = noDebitList.index(str(trueNoDebit))
creditAmount = creditAmountList[noDebitIndex]
summaryDict[summaryCol[2]] = trueNoDebit
summaryDict[summaryCol[3]] = creditAmount

summaryDict

{'Baki Penutup / Closing Balance': '28,433,583.71',
 'Jumlah Debit / Total Debits': '9,352,243.14',
 'Bil. Kredit / No. of Credits': '90',
 'Bil. Debit / No. of Debits': 12816,
 'Jumlah Kredit / Total Credits': '12,298,806.51'}

In [60]:
import xlsxwriter 
  

workbook = xlsxwriter.Workbook(excelFile)
worksheet = workbook.add_worksheet() 

# Rows and columns are zero indexed. 
row = 0

for col in accountInfoDict : 
  
    # write operation perform 
    worksheet.write(row, 0, col + ':')
    worksheet.write(row, 1, accountInfoDict[col])
  
    # incrementing the value of row by one 
    # with each iteratons. 
    row += 1
    
for col in summaryDict : 
  
    # write operation perform 
    worksheet.write(row, 0, col + ':')
    worksheet.write(row, 1, summaryDict[col])
  
    # incrementing the value of row by one 
    # with each iteratons. 
    row += 1
    
workbook.close()

from openpyxl import load_workbook
#write pandas 
writer = pd.ExcelWriter(excelFile, engine='openpyxl')
writer.book = load_workbook(excelFile)
# copy existing sheets
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(excelFile)
df.to_excel(writer, sheet_name='Sheet1', startrow = len(reader) + 2, header = True, index = False)
writer.save()